# Washington DC Biking data | Hourly Bike Count Prediction

# 4. Modeling for Holidays
Rafael Hernandez

In [1]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

In [2]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from dask.distributed import Client

client = Client()
client

<Client: scheduler='tcp://127.0.0.1:57382' processes=4 cores=4>

In [9]:
# Load datasets
we_h = dd.read_csv('./weekends_holi_data_prepared.csv')
we_h['dteday']=dd.to_datetime(we_h.dteday ,unit='ns')

In [10]:
we_h.compute().head()

instant     dteday  yr  holiday  workingday  temp   atemp   hum  windspeed  \
0        7 2011-01-01   0        0           0  0.22  0.2727  0.80     0.0000   
1        8 2011-01-01   0        0           0  0.20  0.2576  0.86     0.0000   
2        9 2011-01-01   0        0           0  0.24  0.2879  0.75     0.0000   
3       10 2011-01-01   0        0           0  0.32  0.3485  0.76     0.0000   
4       11 2011-01-01   0        0           0  0.38  0.3939  0.76     0.2537   

   casual  ...  hr_20  hr_21  hr_22  hr_23  hr_0  hr_1  hr_2  hr_3  hr_4  hr_5  
0       2  ...      0      0      0      0     0     0     0     0     0     0  
1       1  ...      0      0      0      0     0     0     0     0     0     0  
2       1  ...      0      0      0      0     0     0     0     0     0     0  
3       8  ...      0      0      0      0     0     0     0     0     0     0  
4      12  ...      0      0      0      0     0     0     0     0     0     0  

[5 rows x 75 columns]

In [11]:
we_h.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5508 entries, 0 to 5507
Data columns (total 75 columns):
instant                  5508 non-null int64
dteday                   5508 non-null datetime64[ns]
yr                       5508 non-null int64
holiday                  5508 non-null int64
workingday               5508 non-null int64
temp                     5508 non-null float64
atemp                    5508 non-null float64
hum                      5508 non-null float64
windspeed                5508 non-null float64
casual                   5508 non-null int64
registered               5508 non-null int64
cnt                      5508 non-null int64
isDaylight               5508 non-null bool
isNoon                   5508 non-null bool
relative_temp            5508 non-null float64
relative_atemp           5508 non-null float64
relative_hum             5508 non-null float64
relative_windspeed       5508 non-null float64
Wday_RushHour_High       5508 non-null bool
Wday_RushHour_Me

## Subset Data

Use "train_test_split" for dask to subset the train vs test data failed, therefore the . dataset is split manually using the "dteday" from Q1/2011 to Q2/2012 - Train & Q3-Q4/2012 - test.

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(X, y) #Note that does not workl for train_test_split from dask-ml

In [13]:
X_train = we_h[we_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday','instant'],axis=1)
X_train.compute().head()

yr  holiday  workingday  temp   atemp   hum  windspeed  isDaylight  isNoon  \
0   0        0           0  0.22  0.2727  0.80     0.0000       False   False   
1   0        0           0  0.20  0.2576  0.86     0.0000        True   False   
2   0        0           0  0.24  0.2879  0.75     0.0000        True   False   
3   0        0           0  0.32  0.3485  0.76     0.0000        True   False   
4   0        0           0  0.38  0.3939  0.76     0.2537        True   False   

   relative_temp  ...  hr_20  hr_21  hr_22  hr_23  hr_0  hr_1  hr_2  hr_3  \
0      -1.290994  ...      0      0      0      0     0     0     0     0   
1      -2.939874  ...      0      0      0      0     0     0     0     0   
2       0.840027  ...      0      0      0      0     0     0     0     0   
3       6.270697  ...      0      0      0      0     0     0     0     0   
4       4.451044  ...      0      0      0      0     0     0     0     0   

   hr_4  hr_5  
0     0     0  
1     0     0  
2     0     0  
3     0     0  
4     0     0  

[5 rows x 70 columns]

In [14]:
Y_train =we_h['cnt'][we_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)]
Y_train.compute().head()

0     2
1     3
2     8
3    14
4    36
Name: cnt, dtype: int64

In [15]:
x_test = we_h[we_h['dteday'] > datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday','instant'],axis=1)
x_test.compute().head()

yr  holiday  workingday  temp   atemp   hum  windspeed  isDaylight  \
4141   1        0           0  0.78  0.7576  0.66     0.1343       False   
4142   1        0           0  0.76  0.7273  0.70     0.1642       False   
4143   1        0           0  0.76  0.7424  0.72     0.1642       False   
4144   1        0           0  0.74  0.7121  0.74     0.1343       False   
4145   1        0           0  0.74  0.7121  0.74     0.1642       False   

      isNoon  relative_temp  ...  hr_20  hr_21  hr_22  hr_23  hr_0  hr_1  \
4141   False      -0.213977  ...      0      0      0      0     1     0   
4142   False      -0.461837  ...      0      0      0      0     0     1   
4143   False      -0.475006  ...      0      0      0      0     0     0   
4144   False      -0.729215  ...      0      0      0      0     0     0   
4145   False      -0.743564  ...      0      0      0      0     0     0   

      hr_2  hr_3  hr_4  hr_5  
4141     0     0     0     0  
4142     0     0     0     0  
4143     1     0     0     0  
4144     0     1     0     0  
4145     0     0     1     0  

[5 rows x 70 columns]

In [16]:
y_test =we_h['cnt'][we_h['dteday'] > datetime.datetime(2012, 7, 5, 0, 0)]
y_test.compute().head()

4141    150
4142     71
4143     58
4144     31
4145     16
Name: cnt, dtype: int64

In [17]:
X_train = X_train#.values
Y_train = Y_train#.values

x_test = x_test#.values
y_test = y_test#.values

## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

Linear Regression using dask provides error "TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'" with the same code that ran for baseline.

In [18]:
# from sklearn.metrics import r2_score

# #X_Train, Y_train = make_regression()
# lrd = LinearRegression() #Linear regression with dask_ml
# lrd.fit(X_train.values, Y_train.values)
# y_pred = lrd.predict(X_Test.values)
# r2_score(y_test.compute(), y_pred.compute())

### Linear Regression using SKLEARN & Dask Dataframes

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

lm_parameters = {'fit_intercept':[True,False] ,'normalize':[True,False]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv ,return_train_score=True)
with joblib.parallel_backend('dask'):
    lm.fit(X_train, Y_train)
lm.cv_results_
lm_predictions_w = lm.predict(x_test)
lm.score(x_test, y_test)

-1.2401470718961247e+21

### Random Forest using SKLEARN & Dask Dataframes

In [20]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                             "n_jobs": [-1],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)
with joblib.parallel_backend('dask'):
    rf.fit(X_train, Y_train)
rf.cv_results_
rf_predictions_w = rf.predict(x_test)
rf.score(x_test, y_test)


0.8579123073021487